In [1]:
import os
import sys
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
import utils.downloading
from utils.progress.log_progress import log_progress
from utils.files.file_helper import ensure_directory, get_all_files_from_subfolders
import utils.configuration
import numpy as np
import pandas as pd

/Users/bulhakovdmytro/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
config = utils.configuration.Configuration()

In [4]:
HISTORICAL_DATA_FOLDER = config.config['DEFAULT']['FLIGHT_DATA']
WEATHER_DATA_FOLDER = config.config['DEFAULT']['WEATHER_DATA']
WEATHER_API_KEY = config.config['DEFAULT']['NOAA_WEATHER_API_KEY']
WEATHER_CITY_CODES = config.config['DEFAULT']['WEATHER_CITY_CODES']
YEARS_TO_DOWNLOAD = config.config['DEFAULT']['WORKING_YEARS']

In [5]:
def download_weather_data(downloader, city_list, country, year_list, save_folder):
    logging.info('Starting to download weather data')
    for year in year_list:
        download_yearly_weather_data(downloader, city_list, country, year, save_folder)

In [6]:
def download_yearly_weather_data(downloader, city_list, country, year, save_folder):
    start_date = '{}-01-01'.format(year)
    end_date = '{}-01-01'.format(year + 1)
    
    logging.info('Requesting data for {}'.format(year))
    
    error_city_list = []
    
    for city in city_list:
        try:
            logging.info('Requesting data for city: {}'.format(city))
            city_weather = weather_downloader.get_weather_for_city_by_name(city, country, start_date, end_date)
            logging.info('Saving data for city: {}'.format(city))
            save_folder_name = get_folder_for_year(save_folder, year)
            ensure_directory(save_folder_name)

            city_weather.to_csv('{}/{}.csv'.format(save_folder_name, city), index=False)
        except Exception as e:
            logging.info('No weather found for city {}'.format(city))
            error_city_list.append(city)

    logging.info('Error cities: {}'.format(error_city_list))

In [7]:
def get_folder_for_year(folder_to_save, year):
    return '{}/{}'.format(folder_to_save, year)

In [8]:
def get_existing_airports_names(historical_data_folder):
    historical_files_list = get_all_files_from_subfolders(HISTORICAL_DATA_FOLDER)
    
    name_list = set()
    
    for file in historical_files_list:
        file_data = pd.read_csv(file, 
                           dtype={'QUARTER': int, 'FL_NUM': str,
                                 'OP_UNIQUE_CARRIER': str, 'TAIL_NUM': str,
                                 'ORIGIN_AIRPORT_ID': int, 'ORIGIN_AIRPORT_SEQ_ID': int,
                                 'ORIGIN_AIRPORT_SEQ_ID': int, 'ORIGIN_CITY_MARKET_ID': int,
                                 'ORIGIN': str, 'ORIGIN_CITY_NAME': str, 
                                 'DEST_AIRPORT_ID': int, 'DEST_AIRPORT_SEQ_ID': int,
                                 'DEST_CITY_MARKET_ID': int, 'DEST': str,
                                 'DEST_CITY_NAME': str, 'CRS_DEP_TIME': int,
                                 'DEP_DELAY': float, 'ARR_DELAY': float,
                                 'CANCELLED': int, 'CANCELLATION_CODE': str,
                                 'DIVERTED': int, 'CRS_ELAPSED_TIME': str,
                                 'CARRIER_DELAY': float, 'WEATHER_DELAY': float,
                                 'NAS_DELAY': float, 'SECURITY_DELAY': float,
                                 'LATE_AIRCRAFT_DELAY': float
                                 }, 
                           parse_dates=['FL_DATE'])
        unique_city_names = np.unique(file_data['ORIGIN_CITY_NAME'])        
        name_list.update(unique_city_names)
        
    return list(name_list)

In [9]:
weather_downloader = utils.downloading.NOAAWeatherDownloader(WEATHER_API_KEY, WEATHER_CITY_CODES)

In [10]:
city_names = get_existing_airports_names(HISTORICAL_DATA_FOLDER)

In [ ]:
download_weather_data(weather_downloader, city_names, 'US', YEARS_TO_DOWNLOAD, WEATHER_DATA_FOLDER)

INFO:root:Starting to download weather data
INFO:root:Requesting data for 2017
INFO:root:Requesting data for city: Kahului, HI
INFO:root:Saving data for city: Kahului, HI
INFO:root:Requesting data for city: Baltimore, MD


In [ ]:
WEATHER_API_KEY